In [2]:
# Train GaussianNB on each fold (one-by-one baseline). Saves per-fold pipelines & OOF preds.
from pathlib import Path
import numpy as np
import pandas as pd
import joblib
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

ROOT = Path("dreaddit_cv_raw_splits")
FOLDS_DIR = ROOT / "folds_selected"
SAVE_ROOT = Path("Machine learning") / "models" / "gaussiannb"   # per your request
SAVE_ROOT.mkdir(parents=True, exist_ok=True)

fold_dirs = sorted([p for p in FOLDS_DIR.glob("fold_*") if p.is_dir()])
if not fold_dirs:
    raise FileNotFoundError(f"No per-fold directories found in {FOLDS_DIR}. Run folds preparation first.")

oof_rows = []
fold_summaries = []

print("Found fold dirs:", [p.name for p in fold_dirs])

for fd in fold_dirs:
    fold_name = fd.name  # fold_01 etc.
    fold_no = fold_name.split("_")[1]
    # load arrays
    train_X_path = fd / f"fold_{fold_no}_train_selected.npy"
    val_X_path   = fd / f"fold_{fold_no}_val_selected.npy"
    train_csv    = fd / f"fold_{fold_no}_train_selected.csv"
    val_csv      = fd / f"fold_{fold_no}_val_selected.csv"

    if not (train_X_path.exists() and val_X_path.exists() and train_csv.exists() and val_csv.exists()):
        raise FileNotFoundError(f"Missing files for {fold_name} in {fd}")

    X_tr = np.load(train_X_path)
    X_val = np.load(val_X_path)
    df_tr = pd.read_csv(train_csv)
    df_val = pd.read_csv(val_csv)

    y_tr = df_tr['label'].values
    y_val = df_val['label'].values
    idx_val = df_val['orig_index'].values

    print(f"\nFold {fold_no}: X_tr {X_tr.shape}, X_val {X_val.shape}, y_tr counts {np.bincount(y_tr)}, y_val counts {np.bincount(y_val)}")

    # train GaussianNB (no class_weight param for NB)
    clf = GaussianNB()
    clf.fit(X_tr, y_tr)

    # preds
    probs_val = clf.predict_proba(X_val)[:,1]  # probability for class 1
    preds_val = (probs_val >= 0.5).astype(int)

    # per-fold metrics
    acc = accuracy_score(y_val, preds_val)
    prec = precision_score(y_val, preds_val, zero_division=0)
    rec = recall_score(y_val, preds_val, zero_division=0)
    f1 = f1_score(y_val, preds_val, zero_division=0)
    try:
        auc = roc_auc_score(y_val, probs_val)
    except Exception:
        auc = float('nan')

    print(f" Fold {fold_no} metrics -> acc: {acc:.4f}, prec: {prec:.4f}, rec: {rec:.4f}, f1: {f1:.4f}, auc: {auc:.4f}")

    # save fold pipeline (just the classifier here)
    fold_pipe_path = SAVE_ROOT / f"fold_{fold_no}_pipeline.joblib"
    joblib.dump(clf, fold_pipe_path)

    # save OOF preds for this fold
    oof_df = pd.DataFrame({
        "orig_index": idx_val,
        "true_label": y_val,
        "prob_pos": probs_val,
        "pred_label": preds_val
    })
    oof_csv_path = SAVE_ROOT / f"fold_{fold_no}_oof.csv"
    oof_df.to_csv(oof_csv_path, index=False)

    fold_summaries.append({
        "fold": fold_no,
        "train_rows": X_tr.shape[0],
        "val_rows": X_val.shape[0],
        "acc": acc, "precision": prec, "recall": rec, "f1": f1, "auc": auc,
        "pipeline_path": str(fold_pipe_path),
        "oof_csv": str(oof_csv_path)
    })

    # collect rows for global OOF
    for i, idx in enumerate(idx_val):
        oof_rows.append({"orig_index": int(idx), "true_label": int(y_val[i]), "prob_pos": float(probs_val[i]), "pred_label": int(preds_val[i])})

# aggregate OOF
oof_all = pd.DataFrame(oof_rows).sort_values("orig_index").reset_index(drop=True)
oof_all_path = SAVE_ROOT / "oof_predictions.csv"
oof_all.to_csv(oof_all_path, index=False)

# compute overall OOF metrics
y_true = oof_all['true_label'].values
y_pred = oof_all['pred_label'].values
y_prob = oof_all['prob_pos'].values

acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, zero_division=0)
rec = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)
try:
    auc = roc_auc_score(y_true, y_prob)
except Exception:
    auc = float('nan')

metrics = {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1, "auc": auc}
print("\n=== Overall OOF Metrics ===")
print(metrics)

# Save metrics and manifest
import json
metrics_path = SAVE_ROOT / "oof_metrics.json"
with open(metrics_path, "w") as f:
    json.dump(metrics, f, indent=2)

manifest_df = pd.DataFrame(fold_summaries)
manifest_df.to_csv(SAVE_ROOT / "folds_summary.csv", index=False)

print("\nSaved aggregated OOF ->", oof_all_path.resolve())
print("Saved OOF metrics ->", metrics_path.resolve())
print("Saved per-fold pipelines & OOF CSVs ->", SAVE_ROOT.resolve())
print("\nDone. Paste the printed output here and I will prepare the next model (Logistic Regression) when you tell me to continue.")


Found fold dirs: ['fold_01', 'fold_02', 'fold_03', 'fold_04', 'fold_05']

Fold 01: X_tr (457, 34), X_val (115, 34), y_tr counts [221 236], y_val counts [56 59]
 Fold 01 metrics -> acc: 0.7391, prec: 0.6933, rec: 0.8814, f1: 0.7761, auc: 0.8105

Fold 02: X_tr (457, 34), X_val (115, 34), y_tr counts [221 236], y_val counts [56 59]
 Fold 02 metrics -> acc: 0.6696, prec: 0.6207, rec: 0.9153, f1: 0.7397, auc: 0.7778

Fold 03: X_tr (458, 34), X_val (114, 34), y_tr counts [222 236], y_val counts [55 59]
 Fold 03 metrics -> acc: 0.6140, prec: 0.6000, rec: 0.7627, f1: 0.6716, auc: 0.7498

Fold 04: X_tr (458, 34), X_val (114, 34), y_tr counts [222 236], y_val counts [55 59]
 Fold 04 metrics -> acc: 0.7193, prec: 0.7077, rec: 0.7797, f1: 0.7419, auc: 0.7945

Fold 05: X_tr (458, 34), X_val (114, 34), y_tr counts [222 236], y_val counts [55 59]
 Fold 05 metrics -> acc: 0.6140, prec: 0.6000, rec: 0.7627, f1: 0.6716, auc: 0.6878

=== Overall OOF Metrics ===
{'accuracy': 0.6713286713286714, 'precision

In [3]:
# Train Logistic Regression (class_weight='balanced') on each fold (one-by-one)
from pathlib import Path
import numpy as np
import pandas as pd
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import json

ROOT = Path("dreaddit_cv_raw_splits")
FOLDS_DIR = ROOT / "folds_selected"
SAVE_ROOT = Path("Machine learning") / "models" / "logreg"
SAVE_ROOT.mkdir(parents=True, exist_ok=True)

fold_dirs = sorted([p for p in FOLDS_DIR.glob("fold_*") if p.is_dir()])
if not fold_dirs:
    raise FileNotFoundError(f"No per-fold directories found in {FOLDS_DIR}. Run folds preparation first.")

oof_rows = []
fold_summaries = []

print("Found fold dirs:", [p.name for p in fold_dirs])

for fd in fold_dirs:
    fold_name = fd.name  # fold_01 etc.
    fold_no = fold_name.split("_")[1]
    # load arrays & csvs
    train_X_path = fd / f"fold_{fold_no}_train_selected.npy"
    val_X_path   = fd / f"fold_{fold_no}_val_selected.npy"
    train_csv    = fd / f"fold_{fold_no}_train_selected.csv"
    val_csv      = fd / f"fold_{fold_no}_val_selected.csv"

    if not (train_X_path.exists() and val_X_path.exists() and train_csv.exists() and val_csv.exists()):
        raise FileNotFoundError(f"Missing files for {fold_name} in {fd}")

    X_tr = np.load(train_X_path)
    X_val = np.load(val_X_path)
    df_tr = pd.read_csv(train_csv)
    df_val = pd.read_csv(val_csv)

    y_tr = df_tr['label'].values
    y_val = df_val['label'].values
    idx_val = df_val['orig_index'].values

    print(f"\nFold {fold_no}: X_tr {X_tr.shape}, X_val {X_val.shape}, y_tr counts {np.bincount(y_tr)}, y_val counts {np.bincount(y_val)}")

    # Logistic Regression with balanced class weights
    clf = LogisticRegression(penalty='l2', solver='liblinear', C=1.0, max_iter=2000, class_weight='balanced')
    clf.fit(X_tr, y_tr)

    # preds
    probs_val = clf.predict_proba(X_val)[:,1]
    preds_val = (probs_val >= 0.5).astype(int)

    # per-fold metrics
    acc = accuracy_score(y_val, preds_val)
    prec = precision_score(y_val, preds_val, zero_division=0)
    rec = recall_score(y_val, preds_val, zero_division=0)
    f1 = f1_score(y_val, preds_val, zero_division=0)
    try:
        auc = roc_auc_score(y_val, probs_val)
    except Exception:
        auc = float('nan')

    print(f" Fold {fold_no} metrics -> acc: {acc:.4f}, prec: {prec:.4f}, rec: {rec:.4f}, f1: {f1:.4f}, auc: {auc:.4f}")

    # save fold pipeline
    fold_pipe_path = SAVE_ROOT / f"fold_{fold_no}_pipeline.joblib"
    joblib.dump(clf, fold_pipe_path)

    # save per-fold OOF
    oof_df = pd.DataFrame({
        "orig_index": idx_val,
        "true_label": y_val,
        "prob_pos": probs_val,
        "pred_label": preds_val
    })
    oof_csv_path = SAVE_ROOT / f"fold_{fold_no}_oof.csv"
    oof_df.to_csv(oof_csv_path, index=False)

    fold_summaries.append({
        "fold": fold_no,
        "train_rows": X_tr.shape[0],
        "val_rows": X_val.shape[0],
        "acc": acc, "precision": prec, "recall": rec, "f1": f1, "auc": auc,
        "pipeline_path": str(fold_pipe_path),
        "oof_csv": str(oof_csv_path)
    })

    # collect OOF rows
    for i, idx in enumerate(idx_val):
        oof_rows.append({"orig_index": int(idx), "true_label": int(y_val[i]), "prob_pos": float(probs_val[i]), "pred_label": int(preds_val[i])})

# aggregate OOF across folds
oof_all = pd.DataFrame(oof_rows).sort_values("orig_index").reset_index(drop=True)
oof_all_path = SAVE_ROOT / "oof_predictions.csv"
oof_all.to_csv(oof_all_path, index=False)

# compute overall metrics
y_true = oof_all['true_label'].values
y_pred = oof_all['pred_label'].values
y_prob = oof_all['prob_pos'].values

acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, zero_division=0)
rec = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)
try:
    auc = roc_auc_score(y_true, y_prob)
except Exception:
    auc = float('nan')

metrics = {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1, "auc": auc}
print("\n=== Overall OOF Metrics ===")
print(metrics)

# Save metrics and manifest
metrics_path = SAVE_ROOT / "oof_metrics.json"
with open(metrics_path, "w") as f:
    json.dump(metrics, f, indent=2)

manifest_df = pd.DataFrame(fold_summaries)
manifest_df.to_csv(SAVE_ROOT / "folds_summary.csv", index=False)

print("\nSaved aggregated OOF ->", oof_all_path.resolve())
print("Saved OOF metrics ->", metrics_path.resolve())
print("Saved per-fold pipelines & OOF CSVs ->", SAVE_ROOT.resolve())
print("\nDone. Paste the printed output here when complete and I will prepare the next model.")


Found fold dirs: ['fold_01', 'fold_02', 'fold_03', 'fold_04', 'fold_05']

Fold 01: X_tr (457, 34), X_val (115, 34), y_tr counts [221 236], y_val counts [56 59]
 Fold 01 metrics -> acc: 0.6783, prec: 0.7037, rec: 0.6441, f1: 0.6726, auc: 0.8057

Fold 02: X_tr (457, 34), X_val (115, 34), y_tr counts [221 236], y_val counts [56 59]
 Fold 02 metrics -> acc: 0.7217, prec: 0.6957, rec: 0.8136, f1: 0.7500, auc: 0.8208

Fold 03: X_tr (458, 34), X_val (114, 34), y_tr counts [222 236], y_val counts [55 59]
 Fold 03 metrics -> acc: 0.7018, prec: 0.7119, rec: 0.7119, f1: 0.7119, auc: 0.7815

Fold 04: X_tr (458, 34), X_val (114, 34), y_tr counts [222 236], y_val counts [55 59]
 Fold 04 metrics -> acc: 0.7982, prec: 0.7903, rec: 0.8305, f1: 0.8099, auc: 0.8666

Fold 05: X_tr (458, 34), X_val (114, 34), y_tr counts [222 236], y_val counts [55 59]
 Fold 05 metrics -> acc: 0.6930, prec: 0.7069, rec: 0.6949, f1: 0.7009, auc: 0.7676

=== Overall OOF Metrics ===
{'accuracy': 0.7185314685314685, 'precision

In [1]:
# Train Linear SVM (linear kernel) on each fold (one-by-one)
from pathlib import Path
import numpy as np
import pandas as pd
import joblib
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import json

ROOT = Path("dreaddit_cv_raw_splits")
FOLDS_DIR = ROOT / "folds_selected"
SAVE_ROOT = Path("Machine learning") / "models" / "svm"
SAVE_ROOT.mkdir(parents=True, exist_ok=True)

fold_dirs = sorted([p for p in FOLDS_DIR.glob("fold_*") if p.is_dir()])
if not fold_dirs:
    raise FileNotFoundError(f"No per-fold directories found in {FOLDS_DIR}. Run folds preparation first.")

oof_rows = []
fold_summaries = []

print("Found fold dirs:", [p.name for p in fold_dirs])

for fd in fold_dirs:
    fold_name = fd.name  # fold_01 etc.
    fold_no = fold_name.split("_")[1]
    # load arrays & csvs
    train_X_path = fd / f"fold_{fold_no}_train_selected.npy"
    val_X_path   = fd / f"fold_{fold_no}_val_selected.npy"
    train_csv    = fd / f"fold_{fold_no}_train_selected.csv"
    val_csv      = fd / f"fold_{fold_no}_val_selected.csv"

    if not (train_X_path.exists() and val_X_path.exists() and train_csv.exists() and val_csv.exists()):
        raise FileNotFoundError(f"Missing files for {fold_name} in {fd}")

    X_tr = np.load(train_X_path)
    X_val = np.load(val_X_path)
    df_tr = pd.read_csv(train_csv)
    df_val = pd.read_csv(val_csv)

    y_tr = df_tr['label'].values
    y_val = df_val['label'].values
    idx_val = df_val['orig_index'].values

    print(f"\nFold {fold_no}: X_tr {X_tr.shape}, X_val {X_val.shape}, y_tr counts {np.bincount(y_tr)}, y_val counts {np.bincount(y_val)}")

    # SVC with linear kernel and probability estimates (class_weight balanced)
    clf = SVC(kernel='linear', probability=True, class_weight='balanced')
    clf.fit(X_tr, y_tr)

    # preds
    probs_val = clf.predict_proba(X_val)[:,1]
    preds_val = (probs_val >= 0.5).astype(int)

    # per-fold metrics
    acc = accuracy_score(y_val, preds_val)
    prec = precision_score(y_val, preds_val, zero_division=0)
    rec = recall_score(y_val, preds_val, zero_division=0)
    f1 = f1_score(y_val, preds_val, zero_division=0)
    try:
        auc = roc_auc_score(y_val, probs_val)
    except Exception:
        auc = float('nan')

    print(f" Fold {fold_no} metrics -> acc: {acc:.4f}, prec: {prec:.4f}, rec: {rec:.4f}, f1: {f1:.4f}, auc: {auc:.4f}")

    # save fold pipeline
    fold_pipe_path = SAVE_ROOT / f"fold_{fold_no}_pipeline.joblib"
    joblib.dump(clf, fold_pipe_path)

    # save per-fold OOF
    oof_df = pd.DataFrame({
        "orig_index": idx_val,
        "true_label": y_val,
        "prob_pos": probs_val,
        "pred_label": preds_val
    })
    oof_csv_path = SAVE_ROOT / f"fold_{fold_no}_oof.csv"
    oof_df.to_csv(oof_csv_path, index=False)

    fold_summaries.append({
        "fold": fold_no,
        "train_rows": X_tr.shape[0],
        "val_rows": X_val.shape[0],
        "acc": acc, "precision": prec, "recall": rec, "f1": f1, "auc": auc,
        "pipeline_path": str(fold_pipe_path),
        "oof_csv": str(oof_csv_path)
    })

    # collect OOF rows
    for i, idx in enumerate(idx_val):
        oof_rows.append({"orig_index": int(idx), "true_label": int(y_val[i]), "prob_pos": float(probs_val[i]), "pred_label": int(preds_val[i])})

# aggregate OOF across folds
oof_all = pd.DataFrame(oof_rows).sort_values("orig_index").reset_index(drop=True)
oof_all_path = SAVE_ROOT / "oof_predictions.csv"
oof_all.to_csv(oof_all_path, index=False)

# compute overall metrics
y_true = oof_all['true_label'].values
y_pred = oof_all['pred_label'].values
y_prob = oof_all['prob_pos'].values

acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, zero_division=0)
rec = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)
try:
    auc = roc_auc_score(y_true, y_prob)
except Exception:
    auc = float('nan')

metrics = {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1, "auc": auc}
print("\n=== Overall OOF Metrics ===")
print(metrics)

# Save metrics and manifest
metrics_path = SAVE_ROOT / "oof_metrics.json"
with open(metrics_path, "w") as f:
    json.dump(metrics, f, indent=2)

manifest_df = pd.DataFrame(fold_summaries)
manifest_df.to_csv(SAVE_ROOT / "folds_summary.csv", index=False)

print("\nSaved aggregated OOF ->", oof_all_path.resolve())
print("Saved OOF metrics ->", metrics_path.resolve())
print("Saved per-fold pipelines & OOF CSVs ->", SAVE_ROOT.resolve())
print("\nDone. Paste the printed output here and I will prepare the next model when you say so.")


Found fold dirs: ['fold_01', 'fold_02', 'fold_03', 'fold_04', 'fold_05']

Fold 01: X_tr (457, 34), X_val (115, 34), y_tr counts [221 236], y_val counts [56 59]
 Fold 01 metrics -> acc: 0.7130, prec: 0.7167, rec: 0.7288, f1: 0.7227, auc: 0.8052

Fold 02: X_tr (457, 34), X_val (115, 34), y_tr counts [221 236], y_val counts [56 59]
 Fold 02 metrics -> acc: 0.7217, prec: 0.6901, rec: 0.8305, f1: 0.7538, auc: 0.8130

Fold 03: X_tr (458, 34), X_val (114, 34), y_tr counts [222 236], y_val counts [55 59]
 Fold 03 metrics -> acc: 0.7105, prec: 0.7241, rec: 0.7119, f1: 0.7179, auc: 0.7886

Fold 04: X_tr (458, 34), X_val (114, 34), y_tr counts [222 236], y_val counts [55 59]
 Fold 04 metrics -> acc: 0.7456, prec: 0.7143, rec: 0.8475, f1: 0.7752, auc: 0.8336

Fold 05: X_tr (458, 34), X_val (114, 34), y_tr counts [222 236], y_val counts [55 59]
 Fold 05 metrics -> acc: 0.6667, prec: 0.6667, rec: 0.7119, f1: 0.6885, auc: 0.7441

=== Overall OOF Metrics ===
{'accuracy': 0.7115384615384616, 'precision

In [2]:
# Corrected RandomizedSearchCV (no duplicate leakage) + final test evaluation
import numpy as np
import pandas as pd
import joblib, json
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from scipy.stats import randint
import warnings
warnings.filterwarnings("ignore")

ROOT = Path("dreaddit_cv_raw_splits")
SEL_DIR = ROOT / "selected_features"   # earlier selector artifacts
SAVE_DIR = Path("Machine learning") / "models" / "rf_tuned_corrected"
SAVE_DIR.mkdir(parents=True, exist_ok=True)

# --- Load the unique train/test selected arrays that were produced at selection time ---
# These should be the full train/test selected arrays (no duplication).
train_sel_path = SEL_DIR / "X_train_fused_selected.npy"
test_sel_path  = SEL_DIR / "X_test_fused_selected.npy"
train_csv = ROOT / "train_raw_with_clean_text.csv"
test_csv  = ROOT / "test_frozen_raw_with_clean_text.csv"

for p in (train_sel_path, test_sel_path, train_csv, test_csv):
    if not p.exists():
        raise FileNotFoundError(f"Missing required artifact: {p}")

X_train = np.load(train_sel_path)
X_test  = np.load(test_sel_path)

df_train = pd.read_csv(train_csv)
df_test  = pd.read_csv(test_csv)

y_train = df_train['label'].values
y_test  = df_test['label'].values

print("Loaded unique training shape:", X_train.shape, "labels:", y_train.shape)
print("Loaded frozen test shape     :", X_test.shape, "labels:", y_test.shape)

# quick sanity
if X_train.shape[0] != y_train.shape[0]:
    raise ValueError("Train features / labels row mismatch")

# Define search space (slightly tighter than before)
param_dist = {
    "n_estimators": randint(100, 500),
    "max_depth": [None, 6, 8, 10, 12, 16],
    "min_samples_split": randint(2, 10),
    "min_samples_leaf": randint(1, 6),
    "max_features": ['sqrt', 'log2', 0.2, 0.5],
    "bootstrap": [True, False],
    "class_weight": ["balanced"]
}

rf = RandomForestClassifier(random_state=42, n_jobs=-1)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

search = RandomizedSearchCV(
    rf,
    param_distributions=param_dist,
    n_iter=40,               # reasonably quick
    scoring="f1",
    cv=cv,
    n_jobs=-1,
    verbose=2,
    random_state=42,
    return_train_score=True
)

print("\nStarting corrected RandomizedSearchCV on unique train set...\n")
search.fit(X_train, y_train)

best_params = search.best_params_
best_cv_f1 = search.best_score_
print("\nCorrected Best params found:")
print(best_params)
print("\nCorrected Best CV score (F1):", best_cv_f1)

# Save tuning logs and model
joblib.dump(search.best_estimator_, SAVE_DIR / "rf_tuned_model_corrected.joblib")
pd.DataFrame(search.cv_results_).to_csv(SAVE_DIR / "rf_tuned_cv_results_corrected.csv", index=False)
with open(SAVE_DIR / "rf_tuned_best_params_corrected.json", "w") as f:
    json.dump(best_params, f, indent=2)

# Retrain best estimator on full unique training set (X_train, y_train)
best_rf = search.best_estimator_
best_rf.fit(X_train, y_train)

# Evaluate on frozen test set
probs_test = best_rf.predict_proba(X_test)[:,1]
preds_test = (probs_test >= 0.5).astype(int)

acc = accuracy_score(y_test, preds_test)
prec = precision_score(y_test, preds_test, zero_division=0)
rec = recall_score(y_test, preds_test, zero_division=0)
f1 = f1_score(y_test, preds_test, zero_division=0)
try:
    auc = roc_auc_score(y_test, probs_test)
except:
    auc = float('nan')

test_metrics = {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1, "auc": auc}
print("\n=== Final test metrics (tuned RF) ===")
print(test_metrics)

# Save final test metrics and model
with open(SAVE_DIR / "rf_tuned_test_metrics.json", "w") as f:
    json.dump(test_metrics, f, indent=2)

print("\nSaved tuned model and metrics to:", SAVE_DIR)


Loaded unique training shape: (572, 34) labels: (572,)
Loaded frozen test shape     : (143, 34) labels: (143,)

Starting corrected RandomizedSearchCV on unique train set...

Fitting 5 folds for each of 40 candidates, totalling 200 fits

Corrected Best params found:
{'bootstrap': False, 'class_weight': 'balanced', 'max_depth': None, 'max_features': 0.5, 'min_samples_leaf': 5, 'min_samples_split': 2, 'n_estimators': 358}

Corrected Best CV score (F1): 0.7614910786896413

=== Final test metrics (tuned RF) ===
{'accuracy': 0.6923076923076923, 'precision': 0.7083333333333334, 'recall': 0.6891891891891891, 'f1': 0.6986301369863014, 'auc': np.float64(0.7751664708186448)}

Saved tuned model and metrics to: Machine learning\models\rf_tuned_corrected


In [6]:
# Train LightGBM (per-fold OOF) — balanced class handling
from pathlib import Path
import numpy as np
import pandas as pd
import joblib
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import json

ROOT = Path("dreaddit_cv_raw_splits")
FOLDS_DIR = ROOT / "folds_selected"
SAVE_ROOT = Path("Machine learning") / "models" / "lgbm"
SAVE_ROOT.mkdir(parents=True, exist_ok=True)

fold_dirs = sorted([p for p in FOLDS_DIR.glob("fold_*") if p.is_dir()])
if not fold_dirs:
    raise FileNotFoundError(f"No per-fold directories found in {FOLDS_DIR}")

oof_rows = []
fold_summaries = []

print("Found fold dirs:", [p.name for p in fold_dirs])

for fd in fold_dirs:
    fold_name = fd.name
    fold_no = fold_name.split("_")[1]

    # load data
    X_tr = np.load(fd / f"fold_{fold_no}_train_selected.npy")
    X_val = np.load(fd / f"fold_{fold_no}_val_selected.npy")
    df_tr = pd.read_csv(fd / f"fold_{fold_no}_train_selected.csv")
    df_val = pd.read_csv(fd / f"fold_{fold_no}_val_selected.csv")

    y_tr = df_tr['label'].values
    y_val = df_val['label'].values
    idx_val = df_val['orig_index'].values

    print(f"\nFold {fold_no}: X_tr {X_tr.shape}, X_val {X_val.shape}, y_tr counts {np.bincount(y_tr)}, y_val counts {np.bincount(y_val)}")

    # LightGBM classifier (sklearn API)
    clf = LGBMClassifier(
        objective='binary',
        n_estimators=500,
        learning_rate=0.05,
        num_leaves=31,
        class_weight='balanced',   # respects your imbalance strategy
        random_state=42,
        n_jobs=-1
    )

    # fit
    clf.fit(X_tr, y_tr)

    # predict
    probs_val = clf.predict_proba(X_val)[:, 1]
    preds_val = (probs_val >= 0.5).astype(int)

    # metrics
    acc = accuracy_score(y_val, preds_val)
    prec = precision_score(y_val, preds_val, zero_division=0)
    rec = recall_score(y_val, preds_val, zero_division=0)
    f1 = f1_score(y_val, preds_val, zero_division=0)
    try:
        auc = roc_auc_score(y_val, probs_val)
    except Exception:
        auc = float('nan')

    print(f" Fold {fold_no} metrics -> acc: {acc:.4f}, prec: {prec:.4f}, rec: {rec:.4f}, f1: {f1:.4f}, auc: {auc:.4f}")

    # save model and OOF
    joblib.dump(clf, SAVE_ROOT / f"fold_{fold_no}_pipeline.joblib")
    pd.DataFrame({
        "orig_index": idx_val,
        "true_label": y_val,
        "prob_pos": probs_val,
        "pred_label": preds_val
    }).to_csv(SAVE_ROOT / f"fold_{fold_no}_oof.csv", index=False)

    fold_summaries.append({
        "fold": fold_no,
        "train_rows": X_tr.shape[0],
        "val_rows": X_val.shape[0],
        "acc": acc, "precision": prec, "recall": rec, "f1": f1, "auc": auc
    })

    for i, oi in enumerate(idx_val):
        oof_rows.append({
            "orig_index": int(oi),
            "true_label": int(y_val[i]),
            "prob_pos": float(probs_val[i]),
            "pred_label": int(preds_val[i])
        })

# aggregate OOF
oof_all = pd.DataFrame(oof_rows).sort_values("orig_index").reset_index(drop=True)
oof_all_path = SAVE_ROOT / "oof_predictions.csv"
oof_all.to_csv(oof_all_path, index=False)

# overall metrics
y_true = oof_all['true_label'].values
y_pred = oof_all['pred_label'].values
y_prob = oof_all['prob_pos'].values

acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, zero_division=0)
rec = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)
try:
    auc = roc_auc_score(y_true, y_prob)
except Exception:
    auc = float('nan')

metrics = {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1, "auc": auc}
print("\n=== Overall LightGBM OOF Metrics ===")
print(metrics)

# save metrics & manifest
import json
with open(SAVE_ROOT / "oof_metrics.json", "w") as f:
    json.dump(metrics, f, indent=2)
pd.DataFrame(fold_summaries).to_csv(SAVE_ROOT / "folds_summary.csv", index=False)

print("\nSaved aggregated OOF ->", oof_all_path.resolve())
print("Saved OOF metrics ->", (SAVE_ROOT / "oof_metrics.json").resolve())
print("Saved per-fold pipelines & OOF CSVs ->", SAVE_ROOT.resolve())


Found fold dirs: ['fold_01', 'fold_02', 'fold_03', 'fold_04', 'fold_05']

Fold 01: X_tr (457, 34), X_val (115, 34), y_tr counts [221 236], y_val counts [56 59]
[LightGBM] [Info] Number of positive: 236, number of negative: 221
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000842 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3069
[LightGBM] [Info] Number of data points in the train set: 457, number of used features: 34
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

In [7]:
# ============================
# LightGBM Quick RandomizedSearchCV (no leakage) + final test eval
# ============================
import numpy as np
import pandas as pd
import joblib, json, warnings
from pathlib import Path
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from scipy.stats import randint, uniform
from lightgbm import LGBMClassifier

warnings.filterwarnings("ignore")

ROOT = Path("dreaddit_cv_raw_splits")
SEL_DIR = ROOT / "selected_features"   # contains X_train_fused_selected.npy, X_test_fused_selected.npy
SAVE_DIR = Path("Machine learning") / "models" / "lgbm_tuned_quick"
SAVE_DIR.mkdir(parents=True, exist_ok=True)

# load unique train/test selected arrays
train_sel_path = SEL_DIR / "X_train_fused_selected.npy"
test_sel_path  = SEL_DIR / "X_test_fused_selected.npy"
train_csv = ROOT / "train_raw_with_clean_text.csv"
test_csv  = ROOT / "test_frozen_raw_with_clean_text.csv"

for p in (train_sel_path, test_sel_path, train_csv, test_csv):
    if not p.exists():
        raise FileNotFoundError(f"Missing required artifact: {p}")

X_train = np.load(train_sel_path)
X_test  = np.load(test_sel_path)
df_train = pd.read_csv(train_csv)
df_test  = pd.read_csv(test_csv)
y_train = df_train['label'].values
y_test  = df_test['label'].values

print("Train shape:", X_train.shape, "Test shape:", X_test.shape)

# define LGBM and search space
lgb = LGBMClassifier(objective='binary', random_state=42, n_jobs=-1)

param_dist = {
    "n_estimators": randint(100, 600),
    "learning_rate": uniform(0.01, 0.19),   # 0.01 - 0.20
    "num_leaves": randint(15, 128),
    "max_depth": [ -1, 3, 5, 7, 9, 11, 15 ],
    "min_child_samples": randint(5, 60),
    "subsample": uniform(0.6, 0.4),         # bagging_fraction
    "colsample_bytree": uniform(0.4, 0.6),  # feature_fraction
    "reg_alpha": uniform(0.0, 1.0),
    "reg_lambda": uniform(0.0, 1.0),
    # class weight is handled via scale_pos_weight if you prefer; keep default and use class_weight via fit params
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

search = RandomizedSearchCV(
    estimator=lgb,
    param_distributions=param_dist,
    n_iter=50,
    scoring="f1",
    cv=cv,
    n_jobs=-1,
    verbose=2,
    random_state=42,
    return_train_score=True
)

print("\nStarting LightGBM RandomizedSearchCV (50 candidates)...\n")
search.fit(X_train, y_train)

best_params = search.best_params_
best_cv_f1 = search.best_score_
print("\nBest params found:")
print(best_params)
print("\nBest CV score (F1):", best_cv_f1)

# save results and best estimator
joblib.dump(search.best_estimator_, SAVE_DIR / "lgbm_tuned_model.joblib")
pd.DataFrame(search.cv_results_).to_csv(SAVE_DIR / "lgbm_tuned_cv_results.csv", index=False)
with open(SAVE_DIR / "lgbm_tuned_best_params.json", "w") as f:
    json.dump(best_params, f, indent=2)

# retrain best on full training set
best_lgb = search.best_estimator_
best_lgb.fit(X_train, y_train)

# evaluate on frozen test
probs_test = best_lgb.predict_proba(X_test)[:,1]
preds_test = (probs_test >= 0.5).astype(int)

acc = accuracy_score(y_test, preds_test)
prec = precision_score(y_test, preds_test, zero_division=0)
rec = recall_score(y_test, preds_test, zero_division=0)
f1 = f1_score(y_test, preds_test, zero_division=0)
try:
    auc = roc_auc_score(y_test, probs_test)
except:
    auc = float('nan')

test_metrics = {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1, "auc": auc}
print("\n=== Final test metrics (tuned LightGBM) ===")
print(test_metrics)

with open(SAVE_DIR / "lgbm_tuned_test_metrics.json", "w") as f:
    json.dump(test_metrics, f, indent=2)

print("\nSaved tuned LightGBM model & logs to:", SAVE_DIR)


Train shape: (572, 34) Test shape: (143, 34)

Starting LightGBM RandomizedSearchCV (50 candidates)...

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[LightGBM] [Info] Number of positive: 295, number of negative: 277
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000714 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3550
[LightGBM] [Info] Number of data points in the train set: 572, number of used features: 34
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.515734 -> initscore=0.062958
[LightGBM] [Info] Start training from score 0.062958
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b